练习：使用pytorch对线性回归项目进行重构，可以看到有多方便！

步骤：
1. 创建数据集(Dataset|TensorDataset)
2. 构建模型(nn.Linear)
3. 定义损失函数(nn.MSELoss)
4. 定义优化器(torch.optim.SGD)
5. 训练模型(定义数据加载器，前向传播、反向传播、更新参数、清零梯度)
6. 优化与结果输出
```python
"""
    使用 sklearn.datasets.make_regression 方法来构建一个模拟的回归数据集。

    make_regression(
        n_samples,      生成的样本数量，决定了数据集的规模。
        n_features,     生成的特征数量，决定了数据维度。
        n_informative,  对目标变量有影响的特征数量（默认 10）。
        n_targets,      目标变量的数量（默认 1，单输出回归）。
        bias,	        目标变量的偏置（截距），默认 0.0。
        noise,          添加到目标变量的噪声标准差，用于模拟真实世界数据的不完美。
        coef,           如果为 True, 会返回生成数据的真实系数（权重），用于了解特征与目标变量间的真实关系。
        random_state,   随机数生成的种子，确保在多次运行中能够复现相同的结果。
    )
 
    返回:
    - X: 生成的特征矩阵。X 的维度是 (n_samples, n_features)
    - y: 生成的目标变量。y 的维度是(n_samples,) 或 (n_samples, n_targets)
    - coef: 如果在调用时 coef 参数为 True，则还会返回真实系数（权重）。coef 的维度是 (n_features,)
"""
```

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import random
from sklearn.datasets import make_regression

# 定义特征数
n_features = 5

def build_dataset():
	
    noise = random.randint(1, 3)
    bias = 14.5

    X, y, coef = make_regression(
        n_samples=1000,         # 生成的样本数量
        n_features=n_features,  # 生成的特征数量
        bias=bias,              # 目标变量的偏置
        noise=noise,            # 噪声
        coef=True,              # 是否返回真实系数(权重)
        random_state=0          # 随机数种子
    )

    # 数据转换为张量
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    coef = torch.tensor(coef, dtype=torch.float32)
    bias = torch.tensor(bias, dtype=torch.float32)

    return X, y, coef, bias


# 训练函数
def train():
    # 0. 构建模型
    model = nn.Linear(n_features, 1)

    # 1. 构建数据集
    X, y, coef, bias = build_dataset()
    dataset = TensorDataset(X, y)

    # 2. 定义训练参数
    learning_rate = 0.1
    epochs = 50
    batch_size = 16

    # 定义损失函数
    criterion = nn.MSELoss()
    # 定义优化器
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # 3. 开始训练
    for epoch in range(epochs):
        # 4. 构建数据集加载器
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        epoch_loss = 0
        num_batches = 0
        for train_X, train_y in data_loader:
            num_batches += 1
            # 5. 前向传播
            y_pred = model(train_X)
            # 6. 计算损失，注意y_pred, train_y的形状保持一致
            loss = criterion(y_pred, train_y.reshape(-1, 1))
            # 7. 梯度清零
            optimizer.zero_grad()
            # 8. 反向传播：会自动计算梯度
            loss.backward()
            # 9. 更新参数
            optimizer.step()
            # 10. 训练批次及损失率
            epoch_loss += loss.item()

        print(f"Epoch: {epoch}, Loss: {epoch_loss / num_batches}")
        
    # 获取训练好的权重和偏置
    w = model.weight.detach().flatten()  # 将 weight 转换为一维张量
    b = model.bias.detach().item()

    return coef, bias, w, b


if __name__ == "__main__":
    coef, bias, w, b = train()
    print(f"真实系数: {coef}")
    print(f"预测系数: {w}")
    print(f"真实偏置: {bias}")
    print(f"预测偏置: {b}")


Epoch: 0, Loss: 492.10017692238563
Epoch: 1, Loss: 1.0558456996130565
Epoch: 2, Loss: 1.0579621801300654
Epoch: 3, Loss: 1.05587033903788
Epoch: 4, Loss: 1.057352957744447
Epoch: 5, Loss: 1.05511583695336
Epoch: 6, Loss: 1.059192162657541
Epoch: 7, Loss: 1.0545755038185725
Epoch: 8, Loss: 1.0629135639894576
Epoch: 9, Loss: 1.0505398885598258
Epoch: 10, Loss: 1.0632002396242959
Epoch: 11, Loss: 1.0446590719714997
Epoch: 12, Loss: 1.0563169362999143
Epoch: 13, Loss: 1.0743290237964145
Epoch: 14, Loss: 1.0484311604310597
Epoch: 15, Loss: 1.0527783982337466
Epoch: 16, Loss: 1.0640865053449358
Epoch: 17, Loss: 1.05862729038511
Epoch: 18, Loss: 1.0558790853099218
Epoch: 19, Loss: 1.0447073701828244
Epoch: 20, Loss: 1.057395165638318
Epoch: 21, Loss: 1.0567305868580228
Epoch: 22, Loss: 1.0512051038325778
Epoch: 23, Loss: 1.0589572503453208
Epoch: 24, Loss: 1.0568675403557126
Epoch: 25, Loss: 1.0528870087767404
Epoch: 26, Loss: 1.0589996644901851
Epoch: 27, Loss: 1.0637724574596163
Epoch: 28, 